In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.fftpack import fft, ifft
from scipy.fftpack import fftfreq
from matplotlib import animation
plt.style.use('ggplot')
%matplotlib qt

Der indlæses startsbetingelser jævnfør dokumentet:

In [3]:
N = 5000 #antal punkter i xgrid og pgrid
dt = -.1j
timesteps = 15000
dx = 0.1
xl = 0
x = xl + dx * np.arange(N) #xgrid
n = k = np.arange(N) # indeholder indicer, 0,1,2,3,4...
p = fftfreq(N, dx)*2*np.pi

Indlæs startsimpuls:

In [4]:
k0 = 0

Der genereres en gaussisk startbølgefunktionen der passer nogenlunde til det xgrid, der er blevet fastlagt ved dx og N. Også et potentiale.

In [5]:
psi_0 = norm.pdf(x, np.mean(x), 25) * np.exp(-1j*k0*x) #gaussisk starbølgefunktion
Va = np.zeros_like(x)
Va[0:int(len(x)*0.30)] = 200
Va[int(len(x)*0.70):4999] = 200
V = lambda x: Va

Her defineres de deloperationer, der vil blive anvendt i simulationen:

In [6]:
def t_operator(f): #Anvend T-operator delen fra lign 24
    return f * np.exp(-1.j*p**2 *dt/2)

def v_operator(f): #Anvend V-operator delen fra lign 24
    return f * np.exp(-1.j*V(x)*dt)

Endeligt køres koden igennem pr. tidsskridt. Har prøvet at markere den logiske fremgang i kode-kommentarene..

In [7]:
psis = []

psi = psi_0/np.sqrt(np.trapz(np.flip(x),np.abs(psi_0)**2))
f = psi_0/np.sqrt(np.trapz(np.flip(x),np.abs(psi_0)**2))

for t in range(timesteps):
    if t % 40 == 0:
        psis.append(psi) 
    f = t_operator(fft(f)) # f → T_operator(ftilde)
    f = v_operator(ifft(f)) # ftilde → V_operator(f)
    f = t_operator(fft(f)) # f → T_operator(ftilde)
    f = ifft(f) # ftilde → f
    psi = f/np.sqrt(np.trapz(np.flip(x),np.abs(f)**2)) #Normaliser

Plot og animer:

In [8]:
fig, ax = plt.subplots(1,1,tight_layout=True)
psi_plot, = ax.plot([],[],lw=.4,color='k',label=r'$|\Psi|^2$')
ax.set_ylim(0.0,0.05)
ax.legend(frameon=False,fontsize=14)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('x')
ax.set_ylabel(r'Relativ sandsynlighedstæthed $\rho$')
ax.set_title('Simulering af grundtilstanden ved udvikling i imaginær tid')
ax2 = ax.twinx()
ax2.set_ylim(0,350)
ax2.set_yticklabels([])
ax2.show_yticks = False
ax2.plot(x,V(x),color='red',label='V(x)',lw=.4)
ax2.fill_between(x,V(x),color='k',alpha=0.5)
ax2.legend(frameon=False,loc='upper left',fontsize=14)
ax.set_xlim(xl,x[-1])
def update(i):
    psi_plot.set_data(x,np.abs(psis[i])**2)
    return [psi_plot]

anim = animation.FuncAnimation(fig,
                               update,
                               frames=len(psis),
                               interval=2)
anim

In [9]:
writergif = animation.PillowWriter(fps=30)
anim.save('den_endelige_broend_animation.gif',writer=writergif)